# main

In [1]:
from soundcloud import SoundCloud
import requests

import os

from mutagen.id3 import ID3, APIC
import mutagen

def get_resolved(client, track_url):
    res = client.resolve(track_url)
    if res.streamable:
        return res
    else:
        return "weird error bro rip"
    
def get_streaming_url(client, track):
    has_prog = False
    for tr in track.media.transcodings:
        if tr.format.protocol == 'progressive':
            prog_url = tr.url
            has_prog = True
    if has_prog == False:
        print("no progressive streaming found -- download likely broken. will try anyways")
        hls_url = tr.url

    if has_prog:
        headers = client.get_default_headers()
        json = requests.get(prog_url, params={"client_id": client.client_id}, headers=headers)
        stream_url = json.json()["url"]

    else:
        headers = client.get_default_headers()
        json = requests.get(hls_url, params={"client_id": client.client_id}, headers=headers)
        stream_url = json.json()["url"]

    return stream_url

def stream_download(client, stream_url, title):
    stream = requests.get(stream_url)

    os.makedirs('dls', exist_ok=True)
    filepath = os.path.join('dls', f"{title}.mp3")

    with open(filepath, 'wb') as output:
        output.write(stream.content)
        print(f"{title}.mp3, size: {round(os.stat(filepath).st_size / (1024*1024), 2)} mb.")

    return stream_url, filepath

def add_metadata(track, filepath):
    # get cover img, save to 'imgpath'
    cover_img = requests.get(track.artwork_url).content
    os.makedirs('dls', exist_ok=True)
    imgpath = os.path.join('dls', f"coverart.jpg")
    with open(imgpath, 'wb') as img:
        img.write(cover_img)
    
    # add title and artist
    audio_ez = mutagen.File(filepath, easy = True)

    if audio_ez.tags is None:
        audio_ez.add_tags()
    audio_ez['title'] = track.title
    audio_ez['artist'] = track.user.username
    audio_ez.save()

    # add cover art - can't use easyID3

    audio = mutagen.File(filepath)
    with open(imgpath, 'rb') as coverart:
        audio['APIC'] = APIC(
            encoding = 3,
            mime = 'image/jpeg',
            type = 3,
            desc = u'Cover',
            data = coverart.read()
        )
    audio.save()

    os.remove(imgpath)

########################################################################

client_id = 'tPycpzX7dXV3LN9SC9RpDUI9s4lKl9cc'
client = SoundCloud(client_id)
track_url = input("paste pure SC link here (everything before the '?'): ")
res = get_resolved(client, track_url)

_, filepath = stream_download(client, get_streaming_url(client, res), f"{res.user.username} - {res.title}")
add_metadata(res, filepath)


AttributeError: 'NoneType' object has no attribute 'streamable'

In [145]:
print(res.created_at.date())

a = mutagen.File(filepath, easy = True)

a['date'] = res.created_at.date()

2023-11-21


ValueError: Invalid MultiSpec data: datetime.date(2023, 11, 21)

In [122]:
a = mutagen.File(filepath, easy = True)


a.delete()
a.save()

print(a.tags)

None


In [123]:
if a.tags is None:
    print("hi")

print(a.tags)

hi
None


In [89]:
def test_meta(track, filepath):
    audio = mutagen.File(filepath, easy = True)
    if not audio.tags:
        audio.tags = ID3()

    audio['title'] = track.title
    audio['artist'] = track.user.username

    audio.save()
    return audio

In [128]:
aud = add_metadata(res, filepath)

# xtra


In [58]:
def get_streaming_url(client, track):
    has_prog = False
    for tr in track.media.transcodings:
        if tr.format.protocol == 'progressive':
            prog_url = tr.url
            has_prog = True
    if has_prog == False:
        print("no progressive streaming found -- download likely broken. will try anyways")
        hls_url = tr.url

    if has_prog:
        headers = client.get_default_headers()
        json = requests.get(prog_url, params={"client_id": client.client_id}, headers=headers)
        stream_url = json.json()["url"]

    else:
        headers = client.get_default_headers()
        json = requests.get(hls_url, params={"client_id": client.client_id}, headers=headers)
        stream_url = json.json()["url"]

    return stream_url

In [60]:
res = get_resolved(client, 'https://soundcloud.com/jamesivvy/james-ivy-mix-for-skyward-plus')

In [27]:
res = get_resolved(client, 'https://soundcloud.com/perfectlyfine/my-god-its-killing-me')


In [14]:
print(f"{res.title}, {res.user.username}, {res.artwork_url}")

my god its killing me, damageboosting, https://i1.sndcdn.com/artworks-z4gIfrH5ocdCeyuD-KwgzGg-large.jpg


In [61]:
print(get_streaming_url(client, res))

no progressive streaming found -- download likely broken. will try anyways
https://cf-hls-opus-media.sndcdn.com/playlist/3873589f-6ffb-4baf-a7e8-ebcb867ecf2b.64.opus/playlist.m3u8?Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiKjovL2NmLWhscy1vcHVzLW1lZGlhLnNuZGNkbi5jb20vcGxheWxpc3QvMzg3MzU4OWYtNmZmYi00YmFmLWE3ZTgtZWJjYjg2N2VjZjJiLjY0Lm9wdXMvcGxheWxpc3QubTN1OCoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3MDkwODA0MzB9fX1dfQ__&Signature=TGxm-w~R4EBPQKF4f1M8opi~VTIMNbubTKxQXs6O7DzGPpHVPiQyNDISk4Gx3KIgzW28RG2OakWmymKaOkb3w0F7LGqB8Np0~X-azZA9kWVobU7epS6vXokdvGaqIviVLf2UEQrdHC0BM3faN~JvsXCqtK6CkMqSNGLL-5DSJDONyVJkBGJG60R3q7X0EqqRwE5YsS7ZTTEvfxCcMEIE9fQvJp6b2llqC~~6KON1jKt5VmauJKoYMLqHbo41J5X~3rK5CLwprwUG3NDNl33NQMppr0uGgvQMRUTEw5HmCIcBXEL6-n3OBsZDxJrp~haJaEr2UVeVSp3lBFoZv9rZ5Q__&Key-Pair-Id=APKAI6TU7MMXM5DG6EPQ


In [46]:
str = get_streaming_url(client, res)
print(str)

https://cf-hls-opus-media.sndcdn.com/playlist/4b2f8d99-d787-4707-8e6c-f7fad97c7daa.64.opus/playlist.m3u8?Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiKjovL2NmLWhscy1vcHVzLW1lZGlhLnNuZGNkbi5jb20vcGxheWxpc3QvNGIyZjhkOTktZDc4Ny00NzA3LThlNmMtZjdmYWQ5N2M3ZGFhLjY0Lm9wdXMvcGxheWxpc3QubTN1OCoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE3MDkwNzg0NjZ9fX1dfQ__&Signature=aq~89W02A3U5vWgaSS0Jr5Ww6sTpDNOOrhxZmOCfuvBA1frk13hPgeWRq0YoscaqwSTGGb1CTuzuZEf3Yl-UrlRnkdCBQYQkPDIXTTdWIR9yaUidTn0sx7udwXax~cKEmnYd~K2q8H8ZPcuzaft3WvXyP-M0eiEdkFPAucQY1hhZRFZwoPrSzoImZFfi2-SLcvaWLrW5tPLJ0LdXKlqUAnXPnqfYXO73asfo3ECjzxhWSvpcn2bgalcrQ5tCKQcFTUgF4ZyWk9CGDk7OU4Uw7n4p2wIsjdYONoLlHK5cvUcXbAfT1PrHFFmzNwAuOEkStBCJ6V1zbNXCZOv9rjVeMw__&Key-Pair-Id=APKAI6TU7MMXM5DG6EPQ


In [75]:
filepath = "iMGoD_150_MG_OT.mp3"
print(f"export: {round(os.stat(filepath).st_size / (1024*1024), 3)} mb")

export: 3.126 mb


In [130]:
def main(*args):
    for arg in args:
        print("Hello World!")

if __name__ == "__main__":
    main()

# fuckin w objects


In [99]:
from soundcloud import SoundCloud
import requests

import os

from mutagen.id3 import ID3, APIC
import mutagen

class Downloader:
    def __init__(self, client_id):
        self.client = SoundCloud(client_id)

    def get_resolved(self, track_url):
        res = self.client.resolve(track_url)
        if res.streamable:
            return res
        else:
            return "weird error bro rip"
        
    def get_streaming_url(self, track):
        has_prog = False

        for tr in track.media.transcodings:
            if tr.format.protocol == 'progressive':
                prog_url = tr.url
                has_prog = True
                break
        if not has_prog:
            print("no progressive streaming found -- download likely broken. will try anyways")
            hls_url = tr.url

        headers = self.client.get_default_headers()
        if has_prog:
            json = requests.get(prog_url, params={"client_id": self.client.client_id}, headers=headers)
            stream_url = json.json()["url"]
        else:
            json = requests.get(hls_url, params={"client_id": self.client.client_id}, headers=headers)
            stream_url = json.json()["url"]

        return stream_url

    def stream_download(self, stream_url, title):
        stream = requests.get(stream_url)

        os.makedirs('dls', exist_ok=True)
        filepath = os.path.join('dls', f"{title}.mp3")

        with open(filepath, 'wb') as output:
            output.write(stream.content)
            print(f"{title}.mp3, size: {round(os.stat(filepath).st_size / (1024*1024), 2)} mb.")

        return stream_url, filepath

    def add_metadata(self, track, filepath):
        # get cover img, save to 'imgpath'
        cover_img = requests.get(track.artwork_url).content
        os.makedirs('dls', exist_ok=True)
        imgpath = os.path.join('dls', f"coverart.jpg")
        with open(imgpath, 'wb') as img:
            img.write(cover_img)
        
        # add title and artist
        audio_ez = mutagen.File(filepath, easy = True)

        if audio_ez.tags is None:
            audio_ez.add_tags()
        audio_ez['title'] = track.title
        audio_ez['artist'] = track.user.username
        audio_ez.save()

        # add cover art - can't use easyID3

        audio = mutagen.File(filepath)
        with open(imgpath, 'rb') as coverart:
            audio['APIC'] = APIC(
                encoding = 3,
                mime = 'image/jpeg',
                type = 3,
                desc = u'Cover',
                data = coverart.read()
            )
        audio.save()

        os.remove(imgpath)

    def playlist_to_tracks(self, track):
        track_urls = []
        for t in track.tracks:
            try:
                track_urls.append(t.permalink_url)
            except AttributeError:
                track_urls.append(self.client.get_track(t.id).permalink_url)

        return track_urls


########################################################################

client_id = 'tPycpzX7dXV3LN9SC9RpDUI9s4lKl9cc'
downloader = Downloader(client_id)
track_url = input("paste pure SC link here (everything before the '?'): ")
res = downloader.get_resolved(track_url)

_, filepath = downloader.stream_download(downloader.get_streaming_url(res), res.title)
downloader.add_metadata(res, filepath)


AttributeError: 'NoneType' object has no attribute 'streamable'

In [11]:
from mutagen.easyid3 import EasyID3

EasyID3.RegisterTextKey('comment', 'COMM')
print(' '.join(key for key in EasyID3.valid_keys.keys()))

album bpm compilation composer copyright encodedby lyricist length media mood grouping title version artist albumartist conductor arranger discnumber organization tracknumber author albumartistsort albumsort composersort artistsort titlesort isrc discsubtitle language genre date originaldate performer:* musicbrainz_trackid website replaygain_*_gain replaygain_*_peak musicbrainz_artistid musicbrainz_albumid musicbrainz_albumartistid musicbrainz_trmid musicip_puid musicip_fingerprint musicbrainz_albumstatus musicbrainz_albumtype releasecountry musicbrainz_discid asin performer barcode catalognumber musicbrainz_releasetrackid musicbrainz_releasegroupid musicbrainz_workid acoustid_fingerprint acoustid_id comment


In [101]:
SC_URL_PASTED = 'https://soundcloud.com/jays-revenge-301110014/sets/jasperxjay'

if 'set' in SC_URL_PASTED and 'in' not in SC_URL_PASTED:
    print("hi")

# can just do res.kind
    
from soundcloud import SoundCloud
client = SoundCloud('tPycpzX7dXV3LN9SC9RpDUI9s4lKl9cc')
res = client.resolve(SC_URL_PASTED)

hi


In [104]:
res = client.resolve('https://soundcloud.com/travisscott-2/sets/utopia-831875575')
print(res.kind)

playlist


In [96]:
def playlist_to_tracks(client, track):
    track_urls = []
    for t in track.tracks:
        try:
            track_urls.append(t.permalink_url)
        except AttributeError:
            track_urls.append(client.get_track(t.id).permalink_url)

    return track_urls


In [100]:
a = playlist_to_tracks(client, res)

print (a)

['https://soundcloud.com/jays-revenge-301110014/let-loose-jasper', 'https://soundcloud.com/jays-revenge-301110014/i-cant-stop-jasper1taejoon', 'https://soundcloud.com/jays-revenge-301110014/hit-me-back-jasper-saintdadah', 'https://soundcloud.com/jays-revenge-301110014/japanese-jasper', 'https://soundcloud.com/jays-revenge-301110014/dont-recommend-w-mozy-jaspernavi8', 'https://soundcloud.com/jays-revenge-301110014/what-is-u-do-jasper', 'https://soundcloud.com/jays-revenge-301110014/believe-it-jasper', 'https://soundcloud.com/jays-revenge-301110014/either-way', 'https://soundcloud.com/jays-revenge-301110014/the-other-side']


# Playground

In [124]:
from soundcloud import SoundCloud
import requests

import os

from mutagen.id3 import ID3, APIC
import mutagen

class Downloader:
    def __init__(self, client_id):
        self.client = SoundCloud(client_id)

    def get_resolved(self, track_url):
        res = self.client.resolve(track_url)
        try:
            if res.streamable:
                return res
            else:
                return "weird error bro rip"
        except AttributeError:
            return res
        
    def get_streaming_url(self, track):
        has_prog = False

        for tr in track.media.transcodings:
            if tr.format.protocol == 'progressive':
                prog_url = tr.url
                has_prog = True
                break
        if not has_prog:
            print("no progressive streaming found -- download likely broken. will try anyways")
            hls_url = tr.url

        headers = self.client.get_default_headers()
        if has_prog:
            json = requests.get(prog_url, params={"client_id": self.client.client_id}, headers=headers)
            stream_url = json.json()["url"]
        else:
            json = requests.get(hls_url, params={"client_id": self.client.client_id}, headers=headers)
            stream_url = json.json()["url"]

        return stream_url

    def stream_download(self, stream_url, title):
        stream = requests.get(stream_url)

        os.makedirs('dls', exist_ok=True)
        filepath = os.path.join('dls', f"{title}.mp3")

        with open(filepath, 'wb') as output:
            output.write(stream.content)
            print(f"{title}.mp3, size: {round(os.stat(filepath).st_size / (1024*1024), 2)} mb.")

        return stream_url, filepath

    def add_metadata(self, track, filepath):
        # get cover img, save to 'imgpath'
        cover_img = requests.get(track.artwork_url).content
        os.makedirs('dls', exist_ok=True)
        imgpath = os.path.join('dls', f"coverart.jpg")
        with open(imgpath, 'wb') as img:
            img.write(cover_img)
        
        # add title and artist
        audio_ez = mutagen.File(filepath, easy = True)

        if audio_ez.tags is None:
            audio_ez.add_tags()
        audio_ez['title'] = track.title
        audio_ez['artist'] = track.user.username
        audio_ez.save()

        # add cover art - can't use easyID3

        audio = mutagen.File(filepath)
        with open(imgpath, 'rb') as coverart:
            audio['APIC'] = APIC(
                encoding = 3,
                mime = 'image/jpeg',
                type = 3,
                desc = u'Cover',
                data = coverart.read()
            )
        audio.save()

        os.remove(imgpath)
        return audio_ez['title'], audio_ez['artist']

    def playlist_to_tracks(self, track):
        track_urls = []
        for t in track.tracks:
            try:
                track_urls.append(t.permalink_url)
            except AttributeError:
                track_urls.append(self.client.get_track(t.id).permalink_url)

        return track_urls

In [119]:
# run to test on single track
sc_url = "https://soundcloud.com/travisscott-2/travis-scott-hyaena?in=travisscott-2/sets/utopia-831875575"

In [120]:
# run to test on playlist/album
sc_url = "https://soundcloud.com/travisscott-2/sets/utopia-831875575"

In [126]:
downloader = Downloader('tPycpzX7dXV3LN9SC9RpDUI9s4lKl9cc')

init = downloader.get_resolved(sc_url)

urls = []
if res.kind == 'track':
    urls = res.permalink_url
else:
    urls = downloader.playlist_to_tracks(res)

for track_url in urls:
    res = downloader.get_resolved(track_url)
    stream_url = downloader.get_streaming_url(res)
    filename = f"{res.user.username} - {res.title}"
    _, filepath = downloader.stream_download(stream_url, filename)
    


In [127]:
print(res)

print(urls)

AlbumPlaylist(artwork_url='https://i1.sndcdn.com/artworks-dbb1eace-b7be-4b9b-9cf6-9e5149be0e52-0-large.jpg', created_at=datetime.datetime(2023, 7, 28, 6, 53, tzinfo=tzutc()), description=None, duration=4417541, embeddable_by='all', genre='Hip Hop', id=1659692317, kind='playlist', label_name=None, last_modified=datetime.datetime(2024, 2, 3, 5, 38, 20, tzinfo=tzutc()), licence=None, likes_count=67039, permalink='utopia-831875575', permalink_url='https://soundcloud.com/travisscott-2/sets/utopia-831875575', public=True, purchase_title=None, purchase_url=None, release_date='2023-07-28T00:00:00Z', reposts_count=1624, secret_token=None, sharing='public', tag_list='', title='UTOPIA', uri='https://api.soundcloud.com/playlists/1659692317', user_id=40174345, display_date='2023-07-28T00:00:00Z', managed_by_feeds=True, set_type='album', is_album=True, published_at=datetime.datetime(2023, 7, 28, 0, 0, tzinfo=tzutc()), track_count=19, tracks=[BasicTrack(artwork_url='https://i1.sndcdn.com/artworks-KhB